In [105]:
import matplotlib.pyplot as plt # for plotting
import numpy as np # for transformation
import pandas as pd
from PIL import Image
from os.path import exists

import torch # PyTorch package
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer

In [106]:
# Load the JSON data into a python dictionary
train_data = pd.read_json("train_data.json")

#print(train_data)

# Clean out the games that have no reviews
train_df = train_data.dropna(subset=["sentiment"])

# explode the dataset
train_df_expanded=train_df.explode("screenshots", ignore_index=True)
#train_df_expanded

In [79]:
#Exploration and minor cleaning
print(train_data.columns)

print(train_data["sentiment"].value_counts())

dupe = train_data["appid"].value_counts()
if 2 in dupe.values:
    print("There is at least one value with a count of 2.")
else:
    print("No value has a count of 2.")

Index(['appid', 'release', 'title', 'price', 'sentiment', 'reviews',
       'percentage', 'tags', 'screenshots'],
      dtype='object')
Positive                   2053
Very Positive              1469
Mixed                       922
Mostly Positive             811
Overwhelmingly Positive     130
Mostly Negative             107
Negative                     11
Very Negative                 2
Name: sentiment, dtype: int64
No value has a count of 2.


In [94]:
train_df_expanded["images"] = None
for i in range(len(train_df_expanded["screenshots"])):
    screenshot_filename = train_df_expanded["screenshots"][i]
    image_path = f"C:/Users/nicol/OneDrive - KU Leuven/Desktop/python/Advanced Analytics in Business/2/images/{screenshot_filename}"
    if exists(image_path):
        with Image.open(image_path) as image:
            train_df_expanded.at[i, "images"] = image

In [96]:
# Clean out the games that have no image
train_df_expanded = train_df_expanded.dropna(subset=["images"])


In [107]:
transform = transforms.Compose( # composing several transforms together
    [transforms.ToTensor(), # to tensor object
     transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))])

train_df_expanded["images"] = None
for i in range(len(train_df_expanded["screenshots"])):
    screenshot_filename = train_df_expanded["screenshots"][i]
    if exists(image_path):
        with Image.open(image_path) as image:
            transformed_image = transform(image)
            train_df_expanded.at[i, "images"] = transformed_image

# Clean out the games that have no image
train_df_expanded = train_df_expanded.dropna(subset=["images"])

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 24883200 bytes.

In [ ]:
train_df_expanded

In [97]:
# set batch_size
batch_size = 4

# set number of workers
num_workers = 2

# train data
trainloader = torch.utils.data.DataLoader(train_df_expanded, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

# put 10 classes into a set
classes = ('Overwhelmingly Positive', 'Very Positive', 'Positive', 'Mostly Positive',
           'Mixed', 'Mostly Negative', 'Negative', 'Very Negative', 'Ovewhelmingly Negative')